In [ ]:
%run base.ipynb

In [ ]:
experiment = 'new/all-mpnet-zero'
# experiment = 'new/all-mpnet-mulneg'
# experiment = 'new/all-mpnet-mulneg-beginning'
# experiment = 'new/all-mpnet-mulneg-pro_con'

experiment_path = get_experiment_path(experiment)
experiment_path

## Maps 

In [ ]:
import pandas as pd
from pathlib import Path

from kialo_util import read_data

maps = read_data({'local': True, 'debug_maps_size': 10, 'debug_maps_size': None, 'lang': None})

In [ ]:
import pandas as pd

maps_df = pd.DataFrame([{'id': m.id, 'size': m.number_of_children(), 'name': m.name} for m in maps]).set_index('id')
maps_df

### Pro/Con Statistics 

In [ ]:
node_type_stats = {None:[], 1:[], -1:[]}
for m in maps:
    for x in m.child_nodes:
        node_type_stats[None].append(x)
        node_type = x.type
        node_type_stats[node_type].append(x)

In [ ]:
{k:(len(v), len(v)/len(node_type_stats[None])) for k,v in node_type_stats.items()}

## Analyse Node Results 

In [ ]:
import json
all_nodes_results = json.loads((experiment_path / 'results/all/all_nodes.json').read_text())
argument_maps_dict = {str(x.id):{n.id:n for n in x.all_nodes} for x in maps}

In [ ]:
all_nodes_results['2223']['only_leafs_limited_types'][0]

### Pro/Con

In [ ]:
from collections import defaultdict
node_type_results = {None:[], 1:[], -1:[]}
maps_node_type_results = {None: defaultdict(list), 1:defaultdict(list), -1: defaultdict(list)}
for map_id, v in all_nodes_results.items():
    if v['only_leafs_limited_types']:
        for x in v['only_leafs_limited_types']:
            r = (x['rank'], x['id'])
            node_type_results[None].append(r)
            maps_node_type_results[None][map_id].append(r)
            node_type = argument_maps_dict[map_id][x['id']].type
            node_type_results[node_type].append(r)
            maps_node_type_results[node_type][map_id].append(r)

In [ ]:
# from evaluation import Evaluation
# {k:Evaluation.precision_at_rank(v, 5) for k,v in node_type_results.items()}

In [ ]:
import statistics
def pak(l, k):
    return statistics.mean(x[0]<=k for x in l)

print(' & '.join([str(len(v)) for v in node_type_results.values()]))
print(' & '.join([f'{statistics.mean(pak(l, 5) for l in v.values()):.2f}' for k,v in maps_node_type_results.items()]))
print(' & '.join([f'{pak(v, 5):.2f}' for k,v in node_type_results.items()]))

## Annotated Samples Results

In [ ]:
import json
all_samples_results = json.loads((experiment_path / 'results/annotated_samples_predictions.json').read_text())
all_samples_results['17763.90']

### Pro/Con

In [ ]:
from collections import defaultdict
node_type_sample_results = {None:[], 1:[], -1:[]}
for idx, v in all_samples_results.items():
    r = (v['rank'], idx)
    node_type_sample_results[None].append(r)
    node_type_sample_results[v['nodeType']].append(r)

In [ ]:
print(' & '.join([str(k) for k,v in node_type_results.items()]), end=' & ')
print(' & '.join([str(k) for k,v in maps_node_type_results.items()]))

print(' & '.join([str(len(v)) for v in node_type_results.values()]), end=' & ')
print(' & '.join([str(len(v)) for v in node_type_sample_results.values()]))
print(' \t '.join([f'{statistics.mean(pak(l, 5) for l in v.values()):.4f}' for k,v in maps_node_type_results.items()]), end=' \t ')
print(' \t '.join([f'{pak(v, 5):.3f}' for k,v in node_type_sample_results.items()]))